### Performance Metrics

## Success Indicators

You have now been asked to discover what key performance indicators (KPIs) will determine the success of the new design? Use at least completion rate, time spent on each step and error rates. Add any KPIs you might find relevant.

- **Completion Rate:** The proportion of users who reach the final ‘confirm’ step.
- **Time Spent on Each Step:** The average duration users spend on each step.
- **Error Rates:** If there’s a step where users go back to a previous step, it may indicate confusion or an error. You should consider moving from a later step to an earlier one as an error.

## Redesign Outcome

Based on the chosen KPIs, how does the new design’s performance compare to the old one?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# import reusable functions from utils directory
import sys
sys.path.append('../../utils')
import functions

In [3]:
combined_cleaned_df = pd.read_csv('../../data/raw/combined_cleaned_data.csv')
combined_cleaned_df.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [4]:
print(combined_cleaned_df['process_step'].unique())

['step_3' 'step_2' 'step_1' 'start' 'confirm']


In [5]:
combined_cleaned_df['date_time'] = pd.to_datetime(combined_cleaned_df['date_time'], errors='coerce')

# check if any dates couldn't be converted (i.e., they are NaT)
inconsistent_dates = combined_cleaned_df['date_time'].isna().sum()

# If inconsistent_dates > 0, then there are invalid or mismatched date formats
if inconsistent_dates > 0:
    print(f'There are {inconsistent_dates} inconsistent or invalid date formats in the column.')
else:
    print('All dates in the column have the same format.')

print(combined_cleaned_df[['date_time']].head())

All dates in the column have the same format.
            date_time
0 2017-04-17 15:27:07
1 2017-04-17 15:26:51
2 2017-04-17 15:19:22
3 2017-04-17 15:19:13
4 2017-04-17 15:18:04


In [6]:
functions.inspect_dataframe(combined_cleaned_df)

Check the shape (rows, columns):
(744641, 5)

Column names:
Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

Data types:
client_id                int64
visitor_id              object
visit_id                object
process_step            object
date_time       datetime64[ns]
dtype: object

Missing values:
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64


In [7]:
# make a copy
kpi_df = combined_cleaned_df.copy()

# drop columns
kpi_df = kpi_df.drop(columns=['visitor_id', 'visit_id'])

In [8]:
# change the naming of the steps to numeric values to ensure order
replacement_dict_steps = {
    'start' : 0,
    'step_1' : 1,
    'step_2' : 2,
    'step_3' : 3,
    'confirm' : 4
}

kpi_df['process_step'] = kpi_df['process_step'].map(replacement_dict_steps)

# **Completion Rate**

In [9]:
# filter rows where process_step is 'Finish'
clients_finished = kpi_df[kpi_df['process_step'] == 4]

# print(clients_finished)
total_unique_clients = kpi_df['client_id'].nunique()
# find unique client ids that finished
unique_clients_finished = clients_finished['client_id'].nunique()

print(f'Clients who finished the process: {unique_clients_finished} out ouf {total_unique_clients}.')

Clients who finished the process: 81145 out ouf 120157.


**To-do's**:
- [ ] calculate percentage
- [ ] visual representation?

# **Time spent on each step**

In [10]:
# sort df by client_id and date_time
kpi_df = kpi_df.sort_values(by=['client_id', 'date_time'])

# calculate time difference between steps per client
kpi_df['time_diff'] = kpi_df.groupby('client_id')['date_time'].diff()

# drop first row per client (contains no information)
kpi_df = kpi_df.dropna(subset=['time_diff'])
display(kpi_df)

,client_id,process_step,date_time,time_diff
283822,169,1,2017-04-12 20:19:45,0 days 00:00:09
283821,169,2,2017-04-12 20:20:31,0 days 00:00:46
283820,169,3,2017-04-12 20:22:05,0 days 00:01:34
283819,169,4,2017-04-12 20:23:09,0 days 00:01:04
620397,336,0,2017-06-01 07:42:43,0 days 00:15:48
...,...,...,...,...
640076,9999875,1,2017-06-01 22:40:08,0 days 00:00:07
640075,9999875,1,2017-06-01 22:41:28,0 days 00:01:20
640074,9999875,2,2017-06-01 22:41:47,0 days 00:00:19
640073,9999875,3,2017-06-01 22:44:58,0 days 00:03:11


In [11]:
# calculate average time spent on each step
avg_time_per_step = kpi_df.groupby('process_step')['time_diff'].mean()

# convert to total seconds
avg_time_in_seconds = avg_time_per_step.dt.total_seconds()

# round up to the nearest second to remove decimals
avg_time_in_seconds_rounded = np.ceil(avg_time_in_seconds).astype(int)

# convert back to timedelta
avg_time_per_step_rounded = pd.to_timedelta(avg_time_in_seconds_rounded, unit='s')

# re-map process_step index for readability
# avg_time_per_step_rounded.index = avg_time_per_step_rounded.index.map({0: 'Start', 1: 'Step 1', 2: 'Step 2', 3: 'Step 3', 4: 'Finish'})

print('Average time spent on each step:\n')
print(avg_time_per_step_rounded)

Average time spent on each step:

process_step
0   2 days 21:02:52
1   0 days 00:23:31
2   0 days 00:01:05
3   0 days 00:09:49
4   0 days 17:49:27
Name: time_diff, dtype: timedelta64[ns]


**To-do's**:
- [ ] calculate percentage
- [ ] visual representation?
- [ ] Convert to seconds?

# **Error Rates**
_(Constanza's code)_

In [12]:
df = pd.read_csv('../../data/raw/combined_cleaned_data.csv')
df

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
744636,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
744637,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
744638,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
744639,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [13]:
# drop columns
df = df.drop(columns=['visitor_id', 'visit_id'])
# change the naming of the steps to numeric values to ensure order
replacement_dict_steps = {
    'start' : 0,
    'step_1' : 1,
    'step_2' : 2,
    'step_3' : 3,
    'confirm' : 4
}

df['process_step'] = df['process_step'].map(replacement_dict_steps)

In [14]:
# sort df by client_id and date_time
df = df.sort_values(by=['client_id', 'date_time'])

In [15]:
# Detect backward steps (errors) for all clients
df['backward_step'] = df.groupby('client_id')['process_step'].diff() < 0

# Count the number of unique clients who experienced at least one error
clients_with_errors = df[df['backward_step']]['client_id'].nunique()

# Count the total number of errors for each client
errors_per_client = df[df['backward_step']].groupby('client_id').size()

# Total number of errors across all clients
total_errors = df['backward_step'].sum()

# Output results
print(f'Number of clients with at least one error: {clients_with_errors}')
print('Errors per client:')
print(errors_per_client)
print(f'Total number of errors across all clients: {total_errors}')

Number of clients with at least one error: 42424
Errors per client:
client_id
722        2
805        2
832        1
1028       2
1197       1
          ..
9998460    1
9998980    1
9999150    1
9999729    2
9999768    3
Length: 42424, dtype: int64
Total number of errors across all clients: 74980


In [16]:
# add column 'error_count'
df['error_count'] = df['client_id'].map(errors_per_client).fillna(0).astype(int)

In [17]:
# # Crear un DataFrame con los resultados
# results_df = errors_per_client.reset_index()
# results_df.columns = ['client_id', 'number_of_errors']

# # Guardar el DataFrame en un archivo CSV
# results_df.to_csv('errors_per_client.csv', index=False)

# # Mensaje de confirmación
# print('El archivo 'errors_per_client.csv' se ha guardado correctamente.')

# **Redesign Outcome**
Based on the chosen KPIs, how does the new design’s performance compare to the old one?

For the project requirements and instructions for today’s tasks in full, please refer to the project brief. However, in order to keep on track you may refer to the daily goals outlined below:

By the end of day, we recommend you have:

Reviewed KPI and Metrics material.
Discovered what key performance indicators (KPIs) will determine the success of the new design
Use at least completion rate, time spent on each step and error rates. Add any KPIs you might find relevant.
Evaluated how the new design’s performance compare to the old one, given the chosen KPIs (completion rate, time spent on each step and error rates).

Steps:
- [ ] import `merged_final_demo_final_experiment_clients_df`
- [ ] merge with `kpi_df`
- [ ] add column of errors per client
- [ ] clean data if needed

In [18]:
merged_final_demo_final_experiment_clients_df = pd.read_csv('../../data/clean/merged_final_demo_final_experiment_clients_df.csv')

merged_final_demo_final_experiment_clients_df

,client_id,client_tenure_years,client_tenure_months,client_age,gender,num_accounts,balance,calls_last_6_months,logons_last_6_months,variation
0,836976,6,73,60,Unspecified,2,45105.30,6,9,Test
1,2304905,7,94,58,Unspecified,2,110860.30,6,9,Control
2,1439522,5,64,32,Unspecified,2,52467.79,6,9,Test
3,1562045,16,198,49,Male,2,67454.65,3,6,Test
4,5126305,12,145,33,Female,2,103671.75,0,3,Control
...,...,...,...,...,...,...,...,...,...,...
70589,7993686,4,56,38,Unspecified,3,1411062.68,5,5,Unknown
70590,8981690,12,148,31,Male,2,101867.07,6,6,Unknown
70591,333913,16,198,61,Female,2,40745.00,3,3,Unknown
70592,1573142,21,255,68,Male,3,475114.69,4,4,Unknown


In [19]:
kpi_df.head()

,client_id,process_step,date_time,time_diff
283822,169,1,2017-04-12 20:19:45,0 days 00:00:09
283821,169,2,2017-04-12 20:20:31,0 days 00:00:46
283820,169,3,2017-04-12 20:22:05,0 days 00:01:34
283819,169,4,2017-04-12 20:23:09,0 days 00:01:04
620397,336,0,2017-06-01 07:42:43,0 days 00:15:48


In [20]:
merged_df = pd.merge(kpi_df, merged_final_demo_final_experiment_clients_df, on='client_id', how='outer')

functions.inspect_dataframe(merged_df)

Check the shape (rows, columns):
(629543, 13)

Column names:
Index(['client_id', 'process_step', 'date_time', 'time_diff',
       'client_tenure_years', 'client_tenure_months', 'client_age', 'gender',
       'num_accounts', 'balance', 'calls_last_6_months',
       'logons_last_6_months', 'variation'],
      dtype='object')

Data types:
client_id                         int64
process_step                    float64
date_time                datetime64[ns]
time_diff               timedelta64[ns]
client_tenure_years             float64
client_tenure_months            float64
client_age                      float64
gender                           object
num_accounts                    float64
balance                         float64
calls_last_6_months             float64
logons_last_6_months            float64
variation                        object
dtype: object

Missing values:
client_id                    0
process_step              5059
date_time                 5059
time_diff         

In [21]:
functions.inspect_dataframe(df)

Check the shape (rows, columns):
(744641, 5)

Column names:
Index(['client_id', 'process_step', 'date_time', 'backward_step',
       'error_count'],
      dtype='object')

Data types:
client_id         int64
process_step      int64
date_time        object
backward_step      bool
error_count       int64
dtype: object

Missing values:
client_id        0
process_step     0
date_time        0
backward_step    0
error_count      0
dtype: int64


In [24]:
# match datatypes in dfs before merging on `date_time` column to prevent errors
merged_df['date_time'] = pd.to_datetime(merged_df['date_time'], errors='coerce')
df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')

In [25]:
# select columns to merge
df_selected = df[['client_id', 'date_time', 'backward_step', 'error_count']]
merged_kpi_df = pd.merge(merged_df, df_selected, on=['client_id', 'date_time'], how='left')

In [26]:
merged_kpi_df

,client_id,process_step,date_time,time_diff,client_tenure_years,client_tenure_months,client_age,gender,num_accounts,balance,calls_last_6_months,logons_last_6_months,variation,backward_step,error_count
0,169,1.0,2017-04-12 20:19:45,0 days 00:00:09,21.0,262.0,47.0,Male,2.0,501570.72,4.0,4.0,Unknown,False,0.0
1,169,2.0,2017-04-12 20:20:31,0 days 00:00:46,21.0,262.0,47.0,Male,2.0,501570.72,4.0,4.0,Unknown,False,0.0
2,169,3.0,2017-04-12 20:22:05,0 days 00:01:34,21.0,262.0,47.0,Male,2.0,501570.72,4.0,4.0,Unknown,False,0.0
3,169,4.0,2017-04-12 20:23:09,0 days 00:01:04,21.0,262.0,47.0,Male,2.0,501570.72,4.0,4.0,Unknown,False,0.0
4,336,0.0,2017-06-01 07:42:43,0 days 00:15:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631464,9999875,1.0,2017-06-01 22:40:08,0 days 00:00:07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0
631465,9999875,1.0,2017-06-01 22:41:28,0 days 00:01:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0
631466,9999875,2.0,2017-06-01 22:41:47,0 days 00:00:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0
631467,9999875,3.0,2017-06-01 22:44:58,0 days 00:03:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0


In [27]:
# merged_kpi_df: 5682251 rows, 15 columns - before matching date_time
# merged_kpi_df: 631469 rows × 15 columns - after matching date and time

In [ ]:
merged_kpi_df.rename(columns={'backward_step': 'stepped_back'}, inplace=True)

In [29]:
# fix integer column display
int_columns = ['process_step', 'client_tenure_years', 'client_tenure_months', 'client_age',
               'num_accounts', 'calls_last_6_months', 'logons_last_6_months', 'variation', 'error_count']

# convert selected columns to integers (keep NaN values for now)
for col in int_columns:
    merged_kpi_df[col] = pd.to_numeric(merged_kpi_df[col], errors='coerce').astype('Int64')

functions.inspect_dataframe(merged_kpi_df)

Check the shape (rows, columns):
(631469, 15)

Column names:
Index(['client_id', 'process_step', 'date_time', 'time_diff',
       'client_tenure_years', 'client_tenure_months', 'client_age', 'gender',
       'num_accounts', 'balance', 'calls_last_6_months',
       'logons_last_6_months', 'variation', 'backward_step', 'error_count'],
      dtype='object')

Data types:
client_id                         int64
process_step                      Int64
date_time                datetime64[ns]
time_diff               timedelta64[ns]
client_tenure_years               Int64
client_tenure_months              Int64
client_age                        Int64
gender                           object
num_accounts                      Int64
balance                         float64
calls_last_6_months               Int64
logons_last_6_months              Int64
variation                         Int64
backward_step                    object
error_count                       Int64
dtype: object

Missing values: